# Write a Table with synthetic bounding boxes

<div style="display: inline-flex; align-items: center; gap: 10px;">
        <a href="https://colab.research.google.com/github/3lc-ai/3lc-examples/blob/main/tutorials/write-bb-table.ipynb"
        target="_blank"
            style="background-color: transparent; text-decoration: none; display: inline-flex; align-items: center;
            padding: 5px 10px; font-family: Arial, sans-serif;"> <img
            src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="height: 30px;
            vertical-align: middle;box-shadow: none;"/>
        </a> <a href="https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/write-bb-table.ipynb"
            style="text-decoration: none; display: inline-flex; align-items: center; background-color: #ffffff; border:
            1px solid #d1d5da; border-radius: 8px; padding: 2px 10px; color: #333; font-family: Arial, sans-serif;">
            <svg aria-hidden="true" focusable="false" role="img" class="octicon octicon-mark-github" viewBox="0 0 16 16"
            width="20" height="20" fill="#333"
            style="display:inline-block;user-select:none;vertical-align:text-bottom;overflow:visible; margin-right:
            8px;">
                <path d="M8 0c4.42 0 8 3.58 8 8a8.013 8.013 0 0 1-5.45 7.59c-.4.08-.55-.17-.55-.38 0-.27.01-1.13.01-2.2
                0-.75-.25-1.23-.54-1.48 1.78-.2 3.65-.88 3.65-3.95 0-.88-.31-1.59-.82-2.15.08-.2.36-1.02-.08-2.12 0
                0-.67-.22-2.2.82-.64-.18-1.32-.27-2-.27-.68 0-1.36.09-2 .27-1.53-1.03-2.2-.82-2.2-.82-.44 1.1-.16
                1.92-.08 2.12-.51.56-.82 1.28-.82 2.15 0 3.06 1.86 3.75 3.64 3.95-.23.2-.44.55-.51
                1.07-.46.21-1.61.55-2.33-.66-.15-.24-.6-.83-1.23-.82-.67.01-.27.38.01.53.34.19.73.9.82 1.13.16.45.68
                1.31 2.69.94 0 .67.01 1.3.01 1.49 0 .21-.15.45-.55.38A7.995 7.995 0 0 1 0 8c0-4.42 3.58-8 8-8Z"></path>
            </svg> <span style="vertical-align: middle; color: #333;">Open in GitHub</span>
        </a>
</div>

This notebook demonstrates how to write a `tlc.Table` from scratch using a `tlc.TableWriter`.

We show how to format the data and construct the schema in a way that makes the resulting table viewable in the 3LC Dashboard.

If your dataset is already in a common bounding box format, such as COCO or YOLO, it is more convenient to use
`tlc.Table.from_coco(...)` or `tlc.Table.from_yolo(...)` to create a `tlc.Table`.

In [ ]:
from pathlib import Path

DATA_PATH = Path("../data/cats-and-dogs").absolute()

In [ ]:
%pip install --quiet 3lc torchvision

## Setup the TableWriter

First, we need to import the `tlc` library and create a `tlc.TableWriter` object.
We will provide a `tlc.Schema` to the table writer in a later cell.

A column of bounding boxes in 3LC is represented as a dictionary of the form:

```python
{
    "image_width": float,
    "image_height": float,
    "bb_list": [
        {
            "x0": float,  # First "horizontal" coordinate
            "x1": float,  # Second "horizontal" coordinate
            "y0": float,  # First "vertical" coordinate
            "y1": float,  # Second "vertical" coordinate
            "label": str  # Label of the bounding box
        },
        ...
    ]
}
```

In [ ]:
import tlc

bb_schema = tlc.BoundingBoxListSchema(
    label_value_map={0.0: tlc.MapElement("dog"), 1.0: tlc.MapElement("cat")},
    x0_number_role=tlc.NUMBER_ROLE_BB_CENTER_X,
    y0_number_role=tlc.NUMBER_ROLE_BB_CENTER_Y,
    x1_number_role=tlc.NUMBER_ROLE_BB_SIZE_X,
    y1_number_role=tlc.NUMBER_ROLE_BB_SIZE_Y,
    x0_unit="relative",
    y0_unit="relative",
    x1_unit="relative",
    y1_unit="relative",
    include_segmentation=False,
)

schemas = {
    "image": tlc.ImagePath("image"),
    "bounding_boxes": bb_schema
}

table_writer = tlc.TableWriter(
    project_name="Cats and Dogs Bounding Boxes",
    dataset_name="cats-and-dogs-bbs",
    table_name="original",
    column_schemas=schemas,
    if_exists="overwrite",
)

## Create Table Data

Let's load the data to populate the `tlc.Table` with. We have a dictionary with a mapping from image to it's bounding boxes.

The labels are all in XcYcWH relative format, the one specified in the schema. This means each bounding box is defined by its:

    Xc: The x coordinate of the center of the box,
    Yc: The y coordinate of the center of the box,
    W: The width of the bounding box,
    H: The height of the bounding box,
    C: The category index of the bounding box - here 0 means dog and 1 means cat

The coordinates are between 0 and 1, i.e. relative to the image width and height.

In [ ]:
# Each image has a list of bounding boxes
data = {
    'cats/1500.jpg': [[0.527, 0.529, 0.941, 0.938, 1]],
    'cats/1501.jpg': [[0.470, 0.543, 0.866, 0.829, 1]],
    'cats/1502.jpg': [[0.520, 0.537, 0.705, 0.708, 1]],
    'cats/1503.jpg': [[0.591, 0.501, 0.814, 0.992, 1]],
    'cats/1504.jpg': [[0.487, 0.437, 0.819, 0.790, 1]],
    'dogs/1500.jpg': [[0.496, 0.495, 0.948, 0.897, 0]],
    'dogs/1501.jpg': [[0.484, 0.493, 0.308, 0.923, 0]],
    'dogs/1502.jpg': [[0.531, 0.652, 0.487, 0.688, 0]],
    'dogs/1503.jpg': [[0.520, 0.504, 0.945, 0.968, 0]],
    'dogs/1504.jpg': [[0.530, 0.497, 0.929, 0.944, 0]]
}

When populating the `tlc.Table`, we need to convert these boxes to appropriately formatted dictionaries.

In [ ]:
from PIL import Image

table_rows = {
    "image": [],
    "bounding_boxes": []
}

for relative_image_path, bbs in data.items():
    # Prepare full image path
    image_path = str(Path(DATA_PATH, relative_image_path))

    # Prepare bounding boxes
    image = Image.open(image_path)
    image_width, image_height = image.size
    bb_list = [{"x0": bb[0], "y0": bb[1], "x1": bb[2], "y1": bb[3], "label": bb[4]} for bb in bbs]
    boxes = {"image_height": image_height, "image_width": image_width, "bb_list": bb_list}

    # Populate table rows
    table_rows["image"].append(image_path)
    table_rows["bounding_boxes"].append(boxes)

In [ ]:
table_writer.add_batch(table_rows)
table = table_writer.finalize()

# Inspect the data

In [ ]:
# Inspect the first row
table[0]